In [ ]:
import numpy as np
import torch
from scipy.io import loadmat
from cust_plot import plot
from ml import BrainDataset, sample_batch_fn

# Evaluate train/test results

In [ ]:
# Plot training data
fname = 'demo_1.pt'
data = torch.load(f'results/{fname}', map_location=torch.device('cpu'))
plt_data = {k: data[k] for k in ('rewards','success','mu','sig')}
num_fns = 9
plot(plt_data, num_fns, plt_avg=40)

In [ ]:
# Load test data
res = 68
subj = data['test_idx']
sc = loadmat(f'data/subjfiles_SC{res}.mat')
fc = loadmat(f'data/subjfiles_FC{res}.mat')
sc = np.array([sc[f's{str(z+1).zfill(3)}'] for z in subj])
fc = np.array([fc[f's{str(z+1).zfill(3)}'] for z in subj])
euc_dist = loadmat('data/euc_dist.mat')[f'eu{res}']
hubs = np.loadtxt(f'data/hubs_{res}.txt', dtype=np.int, delimiter=',')
regions = np.loadtxt(f'data/regions_{res}.txt', dtype=np.int, delimiter=',')
test_data = BrainDataset(sc, fc, euc_dist, hubs, regions)

In [ ]:
# Testing on 64 samples
fn_weights = [mu[-1] for mu in plt_data['mu']]
test_rewards = np.zeros(len(test_data))
test_success = np.zeros(len(test_data))
for i, (_, sp, pp, sample_idx) in enumerate(test_data):
    pp.fn_weights = fn_weights
    test_rewards[i], test_success[i] = sample_batch_fn(pp, sp, 64, sample_idx)

test_rewards = test_rewards.mean()
test_success = test_success.mean()
print(test_rewards, test_success)